# Install Taipy

To install Taipy, just `pip install` it.

In [1]:
%pip install taipy -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pmdarima -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.5.0
ipykernel        : 6.16.0
ipywidgets       : 8.0.2
jupyter_client   : 7.3.5
jupyter_core     : 4.11.1
jupyter_server   : not installed
jupyterlab       : not installed
nbclient         : 0.6.8
nbconvert        : 7.1.0
nbformat         : 5.6.1
notebook         : 6.4.12
qtconsole        : 5.3.2
traitlets        : 5.4.0


# Import the packages

In [4]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config, Scope
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic_eng.png)

In [5]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [6]:
Gui(page_md).run(dark_mode=False, run_browser=False, port=6001)

 * Server starting on http://127.0.0.1:6001


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [7]:
slider_value = 0
date_value = None
selected_value = None
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={selector}|> <|{selected_value}|>
"""

In [8]:
Gui(control_md).run(dark_mode=False, run_browser=False)

 * Server starting on http://127.0.0.1:5000


Exception in thread Thread-12 (_run_notebook):
Traceback (most recent call last):
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\server.py", line 172, in _run_notebook
    self._ws.run(self._flask, host=self._host, port=self._port, debug=False, use_reloader=False)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\flask_socketio\__init__.py", line 717, in run
    self.wsgi_server.serve_forever()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\baseserver.py", line 398, in serve_forever
    self.start()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\baseserver.py", line 336, in start
 

## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [9]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [10]:
def initialize_case_evolution(data, selected_country='India') -> pd.DataFrame:
    # Aggregation of the dataframe per Country/Region
    country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()
    
    # a country is selected, here India by default
    country_date_df = country_date_df.loc[country_date_df['Country/Region']==selected_country]
    return country_date_df

In [11]:
country_date_df = initialize_case_evolution(data)
country_date_df.head()

C:\Users\jacta\AppData\Local\Temp\ipykernel_25320\4109659252.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
23021,India,2020-01-30,23.746783,78.96288,1.0,0.0,0.0
23022,India,2020-01-31,23.746783,78.96288,1.0,0.0,0.0
23023,India,2020-02-01,23.746783,78.96288,1.0,0.0,0.0
23024,India,2020-02-02,23.746783,78.96288,2.0,0.0,0.0
23025,India,2020-02-03,23.746783,78.96288,3.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in France (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ porperties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [12]:
country_md = "<|{country_date_df}|chart|x=Date|y=Deaths|type=bar|>"

In [13]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=6005)

 * Server starting on http://127.0.0.1:6005


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [14]:
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [15]:
Gui(country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


Exception in thread Thread-14 (_run_notebook):
Traceback (most recent call last):
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\threading.py", line 946, in run


## Style the graph with personalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

The _options_ dictionary will change the opacity of the unselected markers.

These are Plotly properties.

![](img/stack_chart.png)

In [16]:
layout = {'barmode':'stack'}
options = {"unselected":{"marker":{"opacity":0.5}}}
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>"

    self._target(*self._args, **self._kwargs)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\server.py", line 172, in _run_notebook
    self._ws.run(self._flask, host=self._host, port=self._port, debug=False, use_reloader=False)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\flask_socketio\__init__.py", line 717, in run
    self.wsgi_server.serve_forever()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\baseserver.py", line 398, in serve_forever
    self.start()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\baseserver.py", line 336, in start
    self.init_socket()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\pywsgi.py", line 1545, in init_socket
    StreamServer.init_socket(self)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\server.py", line 180, in init_socket
 

In [17]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=6008)

 * Server starting on http://127.0.0.1:6008


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _country_date_df_)
- Add the total number of Recovered (last line of _country_date_df_)
- Add the total number of Confirmed (last line of _country_date_df_)


In [18]:
country_date_df

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
23021,India,2020-01-30,23.746783,78.962880,1.0,0.0,0.0
23022,India,2020-01-31,23.746783,78.962880,1.0,0.0,0.0
23023,India,2020-02-01,23.746783,78.962880,1.0,0.0,0.0
23024,India,2020-02-02,23.746783,78.962880,2.0,0.0,0.0
23025,India,2020-02-03,23.746783,78.962880,3.0,0.0,0.0
...,...,...,...,...,...,...,...
23353,India,2020-12-27,854.924665,3023.983447,10207871.0,9782669.0,147901.0
23354,India,2020-12-28,854.924665,3023.983447,10224303.0,9807569.0,148153.0
23355,India,2020-12-29,854.924665,3023.983447,10244852.0,9834141.0,148439.0
23356,India,2020-12-30,854.924665,3023.983447,10266674.0,9860280.0,148738.0


This is how we can get the total number of Deaths from the dataset for India.

In [19]:
country_date_df.iloc[-1, 6] # gives the number of deaths for India (5 is for recovered and 4 is confirmed)

148738.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [20]:
country_md = """
## Deaths <|{country_date_df.iloc[-1, 6]}|text|>

## Recovered <|{country_date_df.iloc[-1, 5]}|text|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|text|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [21]:
Gui(country_md).run(dark_mode=False, run_browser=False)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


Exception in thread Thread-16 (_run_notebook):
Traceback (most recent call last):
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\server.py", line 172, in _run_notebook
    self._ws.run(self._flask, host=self._host, port=self._port, debug=False, use_reloader=False)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\flask_socketio\__init__.py", line 717, in run
    self.wsgi_server.serve_forever()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\baseserver.py", line 398, in serve_forever
    self.start()
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\baseserver.py", line 336, in start
 

## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.country_date_df' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [22]:
country_lov = sorted(data["Country/Region"].dropna().unique().tolist())
selected_country = "India"

country_md = """
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

    self.socket = self.get_listener(self.address, self.backlog, self.family)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\server.py", line 192, in get_listener
    return _tcp_listener(address, backlog=backlog, reuse_addr=cls.reuse_addr, family=family)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\server.py", line 288, in _tcp_listener
    sock.bind(address)
  File "C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\_socketcommon.py", line 563, in bind
    return self._sock.bind(address)
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted: ('127.0.0.1', 5000)


In [23]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.country_date_df, ...
    # update country_date_df with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.country_date_df = initialize_case_evolution(data, state.selected_country)

In [24]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=6010)

 * Server starting on http://127.0.0.1:6010


## Layout

Use the [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) block to change the page structure. This block creates invisible columns to put text/visual elements in.

Syntax :
```
<|layout|columns=1 1 1 ...|
(first column)

(in second column)

(third column)
(again, third column)

(...)
|>
```

In [25]:
final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>
|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [26]:
Gui(final_country_md).run(dark_mode=False, run_browser=False, port=6013)

 * Server starting on http://127.0.0.1:6013


![](img/layout.png)

# Map

In [27]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths</br>' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [28]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths</br>Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths</br>Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths</br>Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths</br>Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths</br>Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [29]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [30]:
selected_points = []
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [31]:
Gui(map_md).run(dark_mode=False, run_browser=False, port=6017)

 * Server starting on http://127.0.0.1:6017


![](img/carte.png)

# Part and the _render_ property
- Create a [toggle](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/) (works the same as a selector) with a lov of 'Map' an 'Country'
- Create two part blocks that renders or not depending on the value of the toggle
    - To do this, use the fact that in the _render_ property of the part block, Python code can be inserted in `{}`

In [32]:
representation_toggle = ['Map', 'Country']
selected_representation = representation_toggle[0]

In [33]:
main_page = """
<|{selected_representation}|toggle|lov={representation_toggle}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [34]:
Gui(main_page).run(dark_mode=False, run_browser=False, port=6019)

 * Server starting on http://127.0.0.1:6019


![](img/part_render.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [35]:
# initialise variables
selected_scenario = None
scenario_selector = None

first_date = dt.datetime(2020,11,1)

scenario_name = None

result = None

In [36]:
#Config.configure_job_executions(mode="standalone", nb_of_workers=2)

In [37]:

def add_features(data):
    dates = pd.to_datetime(data["Date"])
    data["Months"] = dates.dt.month
    data["Days"] = dates.dt.isocalendar().day
    data["Week"] = dates.dt.isocalendar().week
    data["Day of week"] = dates.dt.dayofweek
    return data

def create_train_data(final_data, date):
    bool_index = pd.to_datetime(final_data['Date']) <= date
    train_data = final_data[bool_index]
    return train_data

def preprocess(initial_data, country, date):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    final_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    final_data = final_data[['Date','Deaths']]
    final_data = add_features(final_data)
    
    train_data = create_train_data(final_data, date)
    return final_data, train_data


def train_arima(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model):
    predictions = model.predict(n_periods=60)
    return np.array(predictions)


def result(final_data, predictions, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(len(predictions))])
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    predictions = pd.concat([pd.Series(dates, name="Date"),
                             pd.Series(predictions, name="Predictions")], axis=1)
    return final_data.merge(predictions, on="Date", how="outer")


def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return predictions

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Node needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [38]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5),
                                              scope=Scope.GLOBAL)

country_cfg = Config.configure_data_node(id="country", default_data="India",
                                         cacheable=True, validity_period=dt.timedelta(days=5))


date_cfg = Config.configure_data_node(id="date", default_data=dt.datetime(2020,10,10),
                                         cacheable=True, validity_period=dt.timedelta(days=5))

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [39]:
final_data_cfg =  Config.configure_data_node(id="final_data",
                                            cacheable=True, validity_period=dt.timedelta(days=5))


train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))


<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [40]:
task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg, date_cfg],
                                           output=[final_data_cfg,train_data_cfg])

### Data Nodes and Task for train_model

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [41]:
model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5), scope=Scope.PIPELINE)

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_arima,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

<img src="img/forecast_arima.svg" alt="drawing" width="500"/>

In [42]:
predictions_cfg = Config.configure_data_node(id="predictions", scope=Scope.PIPELINE)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=model_cfg,
                                      output=predictions_cfg)

### Data Nodes and Task for result

<img src="img/result.svg" alt="drawing" width="500"/>

In [43]:
result_cfg = Config.configure_data_node(id="result", scope=Scope.PIPELINE)

task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[final_data_cfg, predictions_cfg, date_cfg],
                                      output=result_cfg)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [44]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg])

pipeline_arima_cfg = Config.configure_pipeline(id="ARIMA",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## Add more models

<img src="img/pipeline_linear_regression.svg" alt="drawing" width="500"/>

In [45]:
def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return pd.Series(predictions)


task_train_linear_cfg = Config.configure_task(id="task_train_linear",
                                      function=train_linear_regression,
                                      input=train_data_cfg,
                                      output=model_cfg)

task_forecast_linear_cfg = Config.configure_task(id="task_forecast_linear",
                                      function=forecast_linear_regression,
                                      input=[model_cfg, date_cfg],
                                      output=predictions_cfg)

pipeline_linear_regression_cfg = Config.configure_pipeline(id="LinearRegression",
                                               task_configs=[task_train_linear_cfg,
                                                             task_forecast_linear_cfg,
                                                             task_result_cfg])

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [46]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                          pipeline_arima_cfg,
                                                                          pipeline_linear_regression_cfg])

## Creation and submit of scenario

In [47]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-11-29 15:46:49,341][Taipy][INFO] job JOB_task_preprocess_data_18b9c846-55b8-49cd-bb9a-fdc6aacf2776 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_25320\2986109479.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3403.257, Time=0.22 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3834.545, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3555.070, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3711.502, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3992.428, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3395.911, Time=0.21 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3469.260, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3398.025, Time=0.33 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3395.437, Time=0.31 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3395.219, Time=0.28 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3639.930, Time=0.08 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3397.038, Time=0.32 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3595.496, Time=0.12 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

{'PIPELINE_pipeline_preprocessing_e57d8e82-454b-4545-b0a4-729c411e014a': [<taipy.core.job.job.Job at 0x1a1468d5b70>],
 'PIPELINE_ARIMA_fe30412f-4680-4641-8ac1-d02f679390fd': [<taipy.core.job.job.Job at 0x1a1468f4280>,
 'PIPELINE_LinearRegression_ebfeb41f-5e98-4c0a-8473-a8105334f393': [<taipy.core.job.job.Job at 0x1a1491b6830>,
  <taipy.core.job.job.Job at 0x1a1491242b0>]}

In [48]:
scenario.initial_data.read()

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
0,NaN,NaN,NaN,NaN,51526.0,41727.0,2191.0,2021-01-01
1,NaN,NaN,NaN,NaN,58316.0,33634.0,1181.0,2021-01-01
2,NaN,NaN,NaN,NaN,99897.0,67395.0,2762.0,2021-01-01
3,NaN,NaN,NaN,NaN,8117.0,7463.0,84.0,2021-01-01
4,NaN,NaN,NaN,NaN,17568.0,11146.0,405.0,2021-01-01
...,...,...,...,...,...,...,...,...
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31


In [49]:
scenario.train_data.read()

,Date,Deaths,Months,Days,Week,Day of week
0,2020-01-30,0.0,1,4,5,3
1,2020-01-31,0.0,1,5,5,4
2,2020-02-01,0.0,2,6,5,5
3,2020-02-02,0.0,2,7,5,6
4,2020-02-03,0.0,2,1,6,0
...,...,...,...,...,...,...
250,2020-10-06,104555.0,10,2,41,1
251,2020-10-07,105526.0,10,3,41,2
252,2020-10-08,106490.0,10,4,41,3
253,2020-10-09,107416.0,10,5,41,4


In [50]:
scenario.ARIMA.predictions.read()

array([109313.30105592, 110278.58253483, 111243.83572909, 112209.06063951,
       113174.25726692, 114139.42561216, 115104.56567605, 116069.67745942,
       117034.7609631 , 117999.81618792, 118964.8431347 , 119929.84180428,
       120894.81219749, 121859.75431514, 122824.66815808, 123789.55372713,
       124754.41102312, 125719.24004687, 126684.04079922, 127648.81328099,
       128613.55749301, 129578.27343611, 130542.96111112, 131507.62051887,
       132472.25166018, 133436.85453589, 134401.42914681, 135365.97549379,
       136330.49357765, 137294.98339921, 138259.4449593 , 139223.87825876,
       140188.28329841, 141152.66007907, 142117.00860159, 143081.32886677,
       144045.62087546, 145009.88462848, 145974.12012666, 146938.32737083,
       147902.50636181, 148866.65710043, 149830.77958752, 150794.87382391,
       151758.93981043, 152722.9775479 , 153686.98703715, 154650.96827901,
       155614.9212743 , 156578.84602386, 157542.74252851, 158506.61078908,
       159470.4508064 , 1

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [51]:
tp.submit(scenario)

[2022-11-29 15:46:55,088][Taipy][INFO] job JOB_task_preprocess_data_19acba04-ffb6-481c-9ec0-3016617beb0c is skipped.
[2022-11-29 15:46:55,297][Taipy][INFO] job JOB_task_train_51624b44-c04a-4f38-a715-90bc6b771476 is skipped.
[2022-11-29 15:46:55,389][Taipy][INFO] job JOB_task_forecast_567b5985-2ef7-4a6a-9c1d-57b3ab725278 is completed.
[2022-11-29 15:46:55,458][Taipy][INFO] job JOB_task_result_d35771a0-de3d-486d-8a3f-89c26c38ff1d is completed.
[2022-11-29 15:46:55,643][Taipy][INFO] job JOB_task_train_linear_07bb1bdf-c9a0-4adf-8154-8a1e3668e3a2 is skipped.
[2022-11-29 15:46:55,737][Taipy][INFO] job JOB_task_forecast_linear_cdd77acf-b60d-4a42-a10c-71d1907ac5a6 is completed.
[2022-11-29 15:46:55,808][Taipy][INFO] job JOB_task_result_1f845807-5987-4130-83da-94e2f194d04a is completed.


{'PIPELINE_pipeline_preprocessing_e57d8e82-454b-4545-b0a4-729c411e014a': [<taipy.core.job.job.Job at 0x1a14693f790>],
 'PIPELINE_ARIMA_fe30412f-4680-4641-8ac1-d02f679390fd': [<taipy.core.job.job.Job at 0x1a1480d8ee0>,
 'PIPELINE_LinearRegression_ebfeb41f-5e98-4c0a-8473-a8105334f393': [<taipy.core.job.job.Job at 0x1a149497a30>,
  <taipy.core.job.job.Job at 0x1a1480d9780>]}

## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [52]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-11-29 15:46:56,848][Taipy][INFO] job JOB_task_preprocess_data_1fa6d0c9-ca29-49cb-a4e6-9215318f641c is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_25320\2986109479.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3864.276, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=4187.713, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3862.617, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=4017.317, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4407.771, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3864.226, Time=0.04 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3866.190, Time=0.09 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3873.251, Time=0.01 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 0.277 seconds
[2022-11-29 15:46:57,333][Taipy][INFO] job JOB_task_train_f51b1f00-96a4-47a8-b10d-5f7e947a35a5 is completed.
[2022-11-29 15:46:57,429][Taipy][INFO] job JOB_task_forecast_5a2e55b6-64b4-41cf-bd23-b22f71b3bbfb is completed.
[2022-11-29 15:46:57,500][Taipy][INFO] job JOB_task_result_090144f8-608c-41a6-b9ec-1512ccec7830 is completed.
[2022-11-29 15:46:57,

,Date,Deaths,Months,Days,Week,Day of week,Predictions
0,2020-01-22,0.0,1,3,4,2,NaN
1,2020-01-23,0.0,1,4,4,3,NaN
2,2020-01-24,0.0,1,5,4,4,NaN
3,2020-01-25,0.0,1,6,4,5,NaN
4,2020-01-26,0.0,1,7,4,6,NaN
...,...,...,...,...,...,...,...
340,2020-12-27,334533.0,12,7,52,6,NaN
341,2020-12-28,336438.0,12,1,53,0,NaN
342,2020-12-29,340061.0,12,2,53,1,NaN
343,2020-12-30,343783.0,12,3,53,2,NaN


In [53]:
scenario.ARIMA.predictions.read()

array([215495.1020242 , 216178.19525042, 216883.98252799, 217608.85083673,
       218349.76236952, 219104.16295518, 219869.90506087, 220645.18305303,
       221428.47876508, 222218.51573108, 223014.2207056 , 223814.69130993,
       224619.16882909, 225427.01533976, 226237.69447963, 227050.75527824,
       227865.81856217, 228682.56552454, 229500.7281143 , 230320.08095563,
       231140.43455382, 231961.62958275, 232783.53208195, 233606.02941818,
       234429.02689004, 235252.44487315, 236076.21641975, 236900.28524056,
       237724.60400786, 238549.13292875, 239373.83854562, 240198.69272747,
       241023.67182193, 241848.75594221, 242673.9283676 , 243499.17503945,
       244324.48413729, 245149.84572256, 245975.25143897, 246800.69426059,
       247626.16828006, 248451.66853052, 249277.19083584, 250102.73168476,
       250928.28812504, 251753.85767445, 252579.43824593, 253405.02808473,
       254230.62571541, 255056.22989748, 255881.83958792, 256707.45390976,
       257533.07212566, 2

## Simple framework

In [54]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-11-29 15:46:59,542][Taipy][INFO] job JOB_task_preprocess_data_1229b84d-9c8c-428f-bf3a-1bd3a0c085d3 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_25320\2986109479.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3403.257, Time=0.20 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3834.545, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3555.070, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3711.502, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3992.428, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3395.911, Time=0.21 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3469.260, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3398.025, Time=0.34 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3395.437, Time=0.31 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3395.219, Time=0.55 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3639.930, Time=0.09 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3397.038, Time=0.31 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3595.496, Time=0.12 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(1,1,2)(0,0,0)[0]             : 

{'PIPELINE_pipeline_preprocessing_5b2d1a6a-071d-4bb7-b92e-b28492fdc07b': [<taipy.core.job.job.Job at 0x1a149497610>],
 'PIPELINE_ARIMA_794cc936-4efb-4097-93b8-5e4faef85ba3': [<taipy.core.job.job.Job at 0x1a1505ef7f0>,
 'PIPELINE_LinearRegression_9995d5d9-b8e6-4c4d-9f46-03d6881b8d09': [<taipy.core.job.job.Job at 0x1a1520f10f0>,
  <taipy.core.job.job.Job at 0x1a1505b4430>]}

In [55]:
scenario.ARIMA.task_forecast.function

<function __main__.forecast(model)>

In [56]:
scenario.ARIMA.model.read()

ARIMA(order=(1, 1, 2), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [57]:
scenario.pipelines['LinearRegression'].model.read()

LinearRegression()

In [58]:
[s.country.read() for s in tp.get_scenarios()]

['France', 'US', 'India', 'US', 'India', 'India', 'US']

In [59]:
[s.date.read() for s in tp.get_scenarios()]

[datetime.datetime(2020, 11, 1, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0),
 datetime.datetime(2020, 10, 10, 0, 0)]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [60]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

[('SCENARIO_scenario_292a6503-2912-4500-b4f0-7e5b5f718a33', 'test'), ('SCENARIO_scenario_7cdddafc-ba58-4548-bde4-d578c393d0dc', 'First Scenario'), ('SCENARIO_scenario_85ce38e9-df57-4c86-bbdd-d3bbbd1d7225', 'Second Scenario'), ('SCENARIO_scenario_8dd923e3-8cc1-4c72-b9ea-d8a04f1eaabf', 'First Scenario'), ('SCENARIO_scenario_c15f6a8c-99f8-4b21-88b0-855789ae806c', 'Second Scenario'), ('SCENARIO_scenario_c8552ec1-4919-4187-9d4c-cc2c4a6d0e6d', 'Second Scenario'), ('SCENARIO_scenario_f54ba57f-9fbb-4674-8d7b-073e1abb1602', 'First Scenario')] 
 SCENARIO_scenario_85ce38e9-df57-4c86-bbdd-d3bbbd1d7225


In [61]:
result_arima = scenario.pipelines['ARIMA'].result.read()
result_rd = scenario.pipelines['LinearRegression'].result.read()
result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
result

,Date,Deaths_x,Months_x,Days_x,Week_x,Day of week_x,Predictions_x,Deaths_y,Months_y,Days_y,Week_y,Day of week_y,Predictions_y
0,2020-01-30,0.0,1,4,5,3,NaN,0.0,1,4,5,3,NaN
1,2020-01-31,0.0,1,5,5,4,NaN,0.0,1,5,5,4,NaN
2,2020-02-01,0.0,2,6,5,5,NaN,0.0,2,6,5,5,NaN
3,2020-02-02,0.0,2,7,5,6,NaN,0.0,2,7,5,6,NaN
4,2020-02-03,0.0,2,1,6,0,NaN,0.0,2,1,6,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,2020-12-27,147901.0,12,7,52,6,NaN,147901.0,12,7,52,6,NaN
333,2020-12-28,148153.0,12,1,53,0,NaN,148153.0,12,1,53,0,NaN
334,2020-12-29,148439.0,12,2,53,1,NaN,148439.0,12,2,53,1,NaN
335,2020-12-30,148738.0,12,3,53,2,NaN,148738.0,12,3,53,2,NaN


**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [62]:
prediction_md = """
<|layout|columns=1 2 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

Prediction date
<|{first_date}|date|>

<|{selected_country}|selector|lov={country_lov}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths_x|type[1]=bar|y[2]=Predictions_x|y[3]=Predictions_y|>
"""

In [63]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [64]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    scenario.date.write(state.first_date.replace(tzinfo=None))
    tp.submit(scenario)
    actualize_graph(state)

In [65]:
def actualize_graph(state):
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario
    scenario = tp.get(state.selected_scenario)
    result_arima = scenario.pipelines['ARIMA'].result.read()
    result_rd = scenario.pipelines['LinearRegression'].result.read()
    if result_arima is not None and result_rd is not None:
        state.result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
    state.selected_country = scenario.country.read()

In [66]:
Gui(prediction_md).run(dark_mode=False, port=5090)

 * Server starting on http://127.0.0.1:5090


# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [67]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=6066)

 * Server starting on http://127.0.0.1:6066
